# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [8]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [11]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
file_path_list = []
for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*'))
        for f in files :
            file_path_list.append(os.path.abspath(f))

file_path_list           

['/home/workspace/event_data/2018-11-27-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-07-events.csv',
 '/home/workspace/event_data/2018-11-09-events.csv',
 '/home/workspace/event_data/2018-11-19-events.csv',
 '/home/workspace/event_data/2018-11-05-events.csv',
 '/home/workspace/event_data/2018-11-22-events.csv',
 '/home/workspace/event_data/2018-11-16-events.csv',
 '/home/workspace/event_data/2018-11-26-events.csv',
 '/home/workspace/event_data/2018-11-24-events.csv',
 '/home/workspace/event_data/2018-11-29-events.csv',
 '/home/workspace/event_data/2018-11-15-events.csv',
 '/home/workspace/event_data/2018-11-20-events.csv',
 '/home/workspace/event_data/2018-11-06-events.csv',
 '/home/workspace/event_data/2018-11-18-events.csv',
 '/home/workspace/event_data/2018-11-21-events.csv',
 '/home/workspace/event_data/2018-11-10-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv',
 '/home/workspace/event_data/2018-11-02-events

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [12]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [13]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6832


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [14]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [15]:
# TO-DO: Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [16]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




---

### Query-1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [17]:
# Create table:
#  song_session with columns
# - session_id int
# - item_in_session int 
# - artist text 
# - song text 
# - length float


query = "CREATE TABLE IF NOT EXISTS song_session "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, \
                    PRIMARY KEY(session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
# Insert data
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_session (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [19]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
## Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "SELECT artist, song, length FROM song_session WHERE session_id = 338 AND item_in_session = 4"

try:
    songs = session.execute(query)
except Exception as e:
    print(e)

for row in songs:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

---

### Query-2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [20]:
# Create table
#   artist_in_session
# - user_id int
# - session_id int
# - artist text
# - song text
# - item_in_session int 
# - first_name text
# - last_name text

# Partition Key: user_id, session_id
# Clustering Key: item_in_session
# Primary Key: (user_id, session_id), item_in_session


query = "CREATE TABLE IF NOT EXISTS artist_session "
query = query + "(user_id int, session_id int, artist text, song text, item_in_session int, first_name text, last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)  
     

In [21]:
# Insert data
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_session (user_id, session_id, artist, song, item_in_session, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))
       

In [22]:
# Do select
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

query = "SELECT artist, song, first_name, last_name FROM artist_session WHERE user_id = 10 AND session_id = 182"
try:
    artists = session.execute(query)
except Exception as e:
    print(e)

for row in artists:
    print (row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


---

### Query-3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [23]:
#Create table

#   song_user
# - song text
# - user_id int
# - first_name text
# - last_name text


query = "CREATE TABLE IF NOT EXISTS song_user "
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY(song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [24]:
# Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_user (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))


In [25]:
# Do select
# Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "SELECT first_name, last_name FROM song_user WHERE song = 'All Hands Against His Own'"
try:
    users = session.execute(query)
except Exception as e:
    print(e)

for row in users:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [26]:
## TO-DO: Drop the table before closing out the sessions
query = "DROP TABLE song_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query2 = "DROP TABLE artist_session"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
query3 = "DROP TABLE song_user"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()

## OPTIONAL: Question for the reviewer
 
If you have any question about the starter code or your own implementation, please add it in the cell below. 

For example, if you want to know why a piece of code is written the way it is, or its function, or alternative ways of implementing the same functionality, or if you want to get feedback on a specific part of your code or get feedback on things you tried but did not work.

Please keep your questions succinct and clear to help the reviewer answer them satisfactorily. 

> **_Your question_**